# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [13]:
# your code here
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.optimizers import SGD
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from sklearn.model_selection import train_test_split

# 1) Read `tic-tac-toe.csv` into a dataframe
tictac = pd.read_csv('tic-tac-toe.csv')

# 2) Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
tictac.head()

# 3) Convert the categorical values to numeric in all columns.
tictac = tictac.replace({'x' : 1, 'o' : 2, 'b' : 0})

# 4) Separate the inputs and output.
X = tictac.drop('class', axis = 1)
y = tictac['class']

# 5) Normalize the input data.
X_normalized = (X - X.mean() / X.std())

display(X_normalized.head())

C:\Users\Lexo\AppData\Local\Temp\ipykernel_19004\2250003863.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tictac = tictac.replace({'x' : 1, 'o' : 2, 'b' : 0})


,TL,TM,TR,ML,MM,MR,BL,BM,BR
0,-0.537216,-0.399876,-0.537216,-0.399876,0.298013,0.600124,-0.537216,0.600124,0.462784
1,-0.537216,-0.399876,-0.537216,-0.399876,0.298013,0.600124,0.462784,-0.399876,0.462784
2,-0.537216,-0.399876,-0.537216,-0.399876,0.298013,0.600124,0.462784,0.600124,-0.537216
3,-0.537216,-0.399876,-0.537216,-0.399876,0.298013,0.600124,0.462784,-1.399876,-1.537216
4,-0.537216,-0.399876,-0.537216,-0.399876,0.298013,0.600124,-1.537216,0.600124,-1.537216


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [16]:
# your code here

# 1. Split the training and test data.
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# 2. Create a `Sequential` model.
model = Sequential()

# 3. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. 
# Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
fg = X_normalized.shape[1]

model.add(Dense(3, activation='relu', input_dim = fg))
model.add(Dense(4, activation='relu'))
model.add(Dense(2, activation = "softmax"))

# 4. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 5. Fit the training data.
model.fit(X_normalized, y , epochs = 20, verbose = 2)

# 6. Evaluate your neural network model with the test data.

loss, accuracy = model.evaluate(X_test, y_test)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

# 7. Save your model as `tic-tac-toe.model`.
model.save('tictac.keras')


Epoch 1/20


c:\Users\Lexo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 - 1s - 32ms/step - accuracy: 0.6347 - loss: 0.6698
Epoch 2/20
30/30 - 0s - 3ms/step - accuracy: 0.6420 - loss: 0.6619
Epoch 3/20
30/30 - 0s - 4ms/step - accuracy: 0.6461 - loss: 0.6555
Epoch 4/20
30/30 - 0s - 3ms/step - accuracy: 0.6472 - loss: 0.6503
Epoch 5/20
30/30 - 0s - 2ms/step - accuracy: 0.6503 - loss: 0.6454
Epoch 6/20
30/30 - 0s - 3ms/step - accuracy: 0.6503 - loss: 0.6405
Epoch 7/20
30/30 - 0s - 3ms/step - accuracy: 0.6545 - loss: 0.6363
Epoch 8/20
30/30 - 0s - 3ms/step - accuracy: 0.6545 - loss: 0.6322
Epoch 9/20
30/30 - 0s - 3ms/step - accuracy: 0.6555 - loss: 0.6286
Epoch 10/20
30/30 - 0s - 3ms/step - accuracy: 0.6545 - loss: 0.6250
Epoch 11/20
30/30 - 0s - 3ms/step - accuracy: 0.6534 - loss: 0.6217
Epoch 12/20
30/30 - 0s - 3ms/step - accuracy: 0.6555 - loss: 0.6188
Epoch 13/20
30/30 - 0s - 3ms/step - accuracy: 0.6608 - loss: 0.6161
Epoch 14/20
30/30 - 0s - 3ms/step - accuracy: 0.6608 - loss: 0.6132
Epoch 15/20
30/30 - 0s - 3ms/step - accuracy: 0.6618 - loss: 0.6108

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [21]:
# your code here

tttoe = load_model('tictac.keras')

# Make predictions
classes = tttoe.predict(X_test, batch_size=32)
print(classes.shape)
display(classes)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
(192, 2)


array([[0.4115793 , 0.5884207 ],
       [0.34611925, 0.6538808 ],
       [0.3602108 , 0.6397892 ],
       [0.37575096, 0.62424904],
       [0.37767094, 0.622329  ],
       [0.3950649 , 0.60493505],
       [0.11267626, 0.88732374],
       [0.53928554, 0.4607144 ],
       [0.4616673 , 0.5383327 ],
       [0.33782366, 0.6621763 ],
       [0.278361  , 0.721639  ],
       [0.34128544, 0.6587146 ],
       [0.13417779, 0.86582226],
       [0.40170714, 0.5982929 ],
       [0.27094516, 0.7290548 ],
       [0.23302585, 0.76697415],
       [0.42933503, 0.57066494],
       [0.3833343 , 0.61666566],
       [0.5339328 , 0.4660671 ],
       [0.3413057 , 0.65869427],
       [0.30706882, 0.6929312 ],
       [0.3017931 , 0.6982069 ],
       [0.3024223 , 0.69757766],
       [0.39696094, 0.6030391 ],
       [0.4261509 , 0.573849  ],
       [0.44843295, 0.5515671 ],
       [0.2811141 , 0.7188859 ],
       [0.17204174, 0.8279582 ],
       [0.36735088, 0.63264906],
       [0.16208743, 0.8379126 ],
       [0.

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [25]:
# your code here

model_adam = Sequential()

model_adam.add(Dense(3, activation='relu', input_dim = fg))
model_adam.add(Dense(4, activation='relu'))
model_adam.add(Dense(5, activation='relu'))
model_adam.add(Dense(6, activation='relu'))
model_adam.add(Dense(2, activation = "softmax"))

learn_rate = 0.01 
adam_optimizer = tf.keras.optimizers.Adam(learning_rate = learn_rate)

model_adam.compile(optimizer = adam_optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# Train the new model
model_adam.fit(X_normalized, y, epochs = 30, verbose = 2)

# Evaluate the model
loss, accuracy = model_adam.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Epoch 1/30


c:\Users\Lexo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 - 1s - 42ms/step - accuracy: 0.6514 - loss: 0.6552
Epoch 2/30
30/30 - 0s - 2ms/step - accuracy: 0.6534 - loss: 0.6267
Epoch 3/30
30/30 - 0s - 3ms/step - accuracy: 0.6534 - loss: 0.6031
Epoch 4/30
30/30 - 0s - 3ms/step - accuracy: 0.7056 - loss: 0.5689
Epoch 5/30
30/30 - 0s - 3ms/step - accuracy: 0.7088 - loss: 0.5656
Epoch 6/30
30/30 - 0s - 3ms/step - accuracy: 0.7307 - loss: 0.5423
Epoch 7/30
30/30 - 0s - 3ms/step - accuracy: 0.7422 - loss: 0.5346
Epoch 8/30
30/30 - 0s - 3ms/step - accuracy: 0.7474 - loss: 0.5348
Epoch 9/30
30/30 - 0s - 3ms/step - accuracy: 0.7443 - loss: 0.5283
Epoch 10/30
30/30 - 0s - 3ms/step - accuracy: 0.7463 - loss: 0.5221
Epoch 11/30
30/30 - 0s - 3ms/step - accuracy: 0.7432 - loss: 0.5213
Epoch 12/30
30/30 - 0s - 3ms/step - accuracy: 0.7453 - loss: 0.5195
Epoch 13/30
30/30 - 0s - 4ms/step - accuracy: 0.7422 - loss: 0.5133
Epoch 14/30
30/30 - 0s - 3ms/step - accuracy: 0.7484 - loss: 0.5108
Epoch 15/30
30/30 - 0s - 4ms/step - accuracy: 0.7484 - loss: 0.5068

**Which approach(es) did you find helpful to improve your model performance?**

In [26]:
# your answer here
print("The model with adam optimizer was more helpful with better total loss and test accuracy.")

The model with adam optimizer was more helpful with better total loss and test accuracy.
